In [1]:
!export LC_ALL=en_US.UTF-8
!CUDA_VISIBLE_DEVICES=0,1,2,3
from embedding import *

In [2]:
import json
from pprint import pprint
import re
import numpy as np

import torch
try:
    import nltk
except:
    !pip install nltk
    import nltk
    nltk.download('punkt')
    nltk.download('stopwords')

In [3]:
data = json.load(open('../train-v1.1.json'))

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltkStopWords = stopwords.words('english')
punctuations = [',', '?', '.', '-',]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tshrjn/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/tshrjn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [6]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [7]:
X_train_comp_all = []
X_train_comp_with_answer_marked_all = []
X_train_ans_all = []
X_train_comp_answer_label_all = []
Y_train_ques_all = []
invalid = 0
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            invalid=invalid+1
            continue
        a_lab[start:end] = 1
        
              
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            start,end = find_sub_list(answer,passage)
            if start == -1:
                invalid = invalid+1
                continue
            marked_comp = np.zeros(len(passage))
            marked_comp[start:end] = 1
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_all.append(cappedPassage)
            X_train_comp_with_answer_marked_all.append(marked_comp)
            X_train_ans_all.append(answer)
            X_train_comp_answer_label_all.append(a_lab)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1

In [8]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [9]:
wordToTake = 2000
words = findKMostFrequentWords(100000)

In [10]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(2000, 300)

In [11]:
!hostname
print(invalid)

JackSparrow.local
4858


In [12]:
c = list(zip(X_train_comp_all,X_train_ans_all, Y_train_ques_all,X_train_comp_with_answer_marked_all, X_train_comp_answer_label_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_ans_all_shuffled, Y_train_ques_all_shuffled, X_train_comp_with_answer_marked_all_shuffled, X_train_comp_answer_label_shuffled = zip(*c)


In [13]:
examples_to_take_train = 160

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_all_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
X_train_comp_with_answer_marked = X_train_comp_with_answer_marked_all_shuffled[0:examples_to_take_train]
X_train_comp_answer_label = X_train_comp_answer_label_shuffled[0:examples_to_take_train]

In [14]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [15]:
X_train_comp_with_answer_marked[0], max_document_len

(array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 618)

In [16]:
document_tokens = np.full((examples_to_take_train, max_document_len), END_TOKEN,dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

answer_labels_all = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

question_input_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_output_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)


In [17]:
for i in range(examples_to_take_train):
    answer_labels_all[i,0:len(X_train_comp_answer_label[i])] = X_train_comp_answer_label[i]
    answer_labels[i, 0:len(X_train_comp_with_answer_marked[i])] = X_train_comp_with_answer_marked[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    answer_lengths[i] = len(X_train_ans[i])
    
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)    

In [18]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [19]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

(2000, 300)
5959
406
5800


In [20]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [21]:
def context_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [22]:
document_tokens = context_to_indices_glove(X_train_comp, max_document_len)

In [23]:
max_document_len

618

In [24]:
max_question_len

29

## Batch Data Preparation

In [25]:
import math

def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    
    num_batches = math.ceil(len(inputs[0])/batch_size)

    for i in range(num_batches-1):
        start = 0
        
        output = {'document_tokens':[],
                    'document_lengths':[],
                    'answer_labels_all':[],
                    'answer_labels':[],
                    'answer_lengths': [],
                    'question_input_tokens':[],
                    'question_output_tokens':[],
                    'question_lengths':[]}
        
        for index,inp in enumerate(inputs):
            #maxD = max(inputs[1][start:start+batch_size])
            maxD = max_document_len
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max_question_len
            
            if index == 0:
                output['document_tokens'].append(inp[start:start+batch_size,0:maxD])
            elif index==1:
                output['document_lengths'].append(inp[start:start+batch_size])
            elif index==2:
                output['answer_labels_all'].append(inp[start:start+batch_size,0:maxD])
            elif index==3:
                output['answer_labels'].append(inp[start:start+batch_size,0:maxD])
            elif index==4:
                output['answer_lengths'].append(inp[start:start+batch_size])
            elif index==5:
                output['question_input_tokens'].append(inp[start:start+batch_size, 0:maxQ])
            elif index==6:
                output['question_output_tokens'].append(inp[start:start+batch_size, 0:maxQ])
            elif index==7:
                output['question_lengths'].append(inp[start:start+batch_size])
        
        output["document_tokens"] = np.array(output["document_tokens"])
        output["document_lengths"] = np.array(output["document_lengths"])
        output["answer_labels_all"] = np.array(output["answer_labels_all"])
        output["answer_labels"] = np.array(output["answer_labels"])
        output["answer_lengths"] = np.array(output["question_lengths"])
        output["question_input_tokens"] = np.array(output["question_input_tokens"])
        output["question_output_tokens"] = np.array(output["question_output_tokens"])
        output["question_lengths"] = np.array(output["question_lengths"])
        outputs.append(output)
        start = start + batch_size
        
    # Remaining training sample i.e. training mod batch_size
    #maxD = max(inputs[1][start:])
    maxD = max_document_len
    maxA = max(inputs[4][start:])
    maxQ = max_question_len
    output = {'document_tokens':[],
                'document_lengths':[],
                'answer_labels_all':[],
                'answer_labels':[],
                'answer_lengths': [],
                'question_input_tokens':[],
                'question_output_tokens':[],
                'question_lengths':[]}
    if index == 0:
        output['document_tokens'].append(inp[start:,0:maxD])
    elif index==1:
        output['document_lengths'].append(inp[start:])
    elif index==2:
        output['answer_labels_all'].append(inp[start:,0:maxD])
    elif index==3:
        output['answer_labels'].append(inp[start:,0:maxD])
    elif index==4:
        output['answer_lengths'].append(inp[start:])
    elif index==5:
        output['question_input_tokens'].append(inp[start:, 0:maxQ])
    elif index==6:
        output['question_output_tokens'].append(inp[start:, 0:maxQ])
    elif index==7:
        output['question_lengths'].append(inp[start:])

    output["document_tokens"] = np.array(output["document_tokens"])
    output["document_lengths"] = np.array(output["document_lengths"])
    output["answer_labels"] = np.array(output["answer_labels"])
    output["answer_labels_all"] = np.array(output["answer_labels_all"])
    output["answer_lengths"] = np.array(output["question_lengths"])
    output["question_input_tokens"] = np.array(output["question_input_tokens"])
    output["question_output_tokens"] = np.array(output["question_output_tokens"])
    output["question_lengths"] = np.array(output["question_lengths"])

    outputs.append(output)
    
    return outputs


In [26]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels_all,answer_labels,answer_lengths,question_input_tokens,question_output_tokens,question_lengths]
                    ,batch_size)
for b in batch_input:
    for k, v in b.items():
        b[k] = v.squeeze()
number_of_batches = len(batch_input)
print("Number of batches = ", number_of_batches)

Number of batches =  5


In [27]:
batch_input[1]['answer_labels'][0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Model

In [28]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [29]:
use_cuda = torch.cuda.is_available()
use_attention = False

In [30]:
class Embedder(nn.Module):
    def __init__(self, input_size, output_size):
        super(Embedder, self).__init__()
        self.embedding = nn.Embedding(input_size, output_size)
        self.embedding.weight = nn.Parameter(torch.from_numpy(reduced_glove).float())
        self.embedding.weight.requires_grad = False
    def forward(self, x):
        return self.embedding(x)

In [31]:
class AnswerEncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(AnswerEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True, bidirectional=True) #Input_size = Hidden_Size
        if use_cuda:
            self.gru = self.gru.cuda()
        self.fc = nn.Linear(hidden_size*2, 1)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        final_output = self.fc(output)
        final_output = F.sigmoid(final_output)
        self.hiddenState = hidden
        return final_output, output, hidden
    
    def initHidden(self):
        result = Variable(torch.zeros(2, 32, self.hidden_size)) #2 for BiDirectional
        if use_cuda:
            result = result.cuda()
        return result

In [32]:
class QuestionEncoderRNN(nn.Module):
    
    def __init__(self,input_size, hidden_size):
        super(QuestionEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True)
        if use_cuda:
            self.gru = self.gru.cuda()
    
    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        self.hiddenState = hidden
        return output, hidden
    
    def initHidden(self):
        #result = Variable(torch.zeros(1, batch_size, self.hidden_size))
        result = Variable(torch.zeros(1, 32, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result

In [33]:
class QuestionDecoderRNN(nn.Module):
    
    def __init__(self,input_size, hidden_size):
        super(QuestionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True)
        if use_cuda:
            self.gru = self.gru.cuda()
        
    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        self.hiddenState = hidden
        return output, hidden
    
    def initHidden(self):
        #result = Variable(torch(1, batch_size, self.hidden_size))
        result = Variable(torch.zeros(1, 32, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result
    
'''
class FCLayer(nn.Module):
    def __init__(self,input_size, output_size):
        super(FCLayer, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        return self.fc(x)
'''

class QuestionGenerationFC(nn.Module):
    def __init__(self,input_size, output_size):
        super(QuestionGenerationFC, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, x):
        output = self.fc(x)
        output = F.log_softmax(output, dim=-1)
        return output

In [34]:
class AttnDecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, max_document_len, dropout_p=0.1):
        
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.dropout_p = dropout_p
        self.max_document_len = max_document_len

        self.attn_reduce = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.attn_combine = nn.Linear(self.max_document_len + self.hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        
        if use_cuda:
            self.attn_reduce = self.attn_reduce.cuda()
            self.attn_combine = self.attn_combine.cuda()
            self.dropout = self.dropout.cuda()
        

    def forward(self, input, hidden, encoder_outputs):

        hidden = hidden.squeeze(0)
        hidden = self.dropout(hidden)
        concat = torch.cat((input.unsqueeze(0), hidden.unsqueeze(0)), 1)
        concat_reduced = self.attn_reduce(concat)
        attn_weights = F.softmax(concat_reduced, dim=1)
        attn_applied = torch.mm(encoder_outputs, attn_weights.squeeze(0).unsqueeze(1))
        output = torch.cat((hidden.unsqueeze(0), attn_applied.squeeze(1).unsqueeze(0)), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        self.hidden_state = hidden
        return output, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result

In [35]:
hidden_size = reduced_glove.shape[1]

embedder = Embedder(input_size = reduced_glove.shape[0], output_size = reduced_glove.shape[1])
#fcLayer = FCLayer(hidden_size, hidden_size)
answerEncoder = AnswerEncoderRNN(input_size = hidden_size, hidden_size=int(hidden_size/2))
questionEncoder1 = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionEncoder2 = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)

questionDecoder = QuestionDecoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionGenerator = QuestionGenerationFC(input_size = hidden_size, output_size=reduced_glove.shape[0])
attention = AttnDecoderRNN(hidden_size= hidden_size,max_document_len = max_document_len)

answerEncoder.train()
questionEncoder1.train()
questionEncoder2.train()
questionDecoder.train()
questionGenerator.train()
attention.train()

train_param = []

for model in [answerEncoder, questionEncoder1, questionEncoder2, questionDecoder, questionGenerator, attention]:
    train_param += [p for p in model.parameters() if p.requires_grad]

print("Number of trainable parameters = ", len(train_param))

optimizer = torch.optim.Adam(train_param, 0.01)
criterion1 = nn.BCELoss()
#criterion2 = nn.CrossEntropyLoss()
criterion2 = nn.NLLLoss()
#criterion2 = nn.MultiLabelSoftMarginLoss()

Number of trainable parameters =  28


In [36]:
def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

In [39]:
verboseBatchPrinting = True
averageBatchLossPrinting = True

num_epochs = 100
answer_encoder_hidden = answerEncoder.initHidden()
question_encoder_1_hidden = questionEncoder1.initHidden()
question_encoder_2_hidden = questionEncoder2.initHidden()
question_decoder_hidden = None

Attention_Weights = None
attn_output = None
attention_hidden = attention.initHidden()

for epoch in range(1, num_epochs+1):
    avg_loss = 0
    
    for batch_num in range(number_of_batches):
        
        answer_encoder_hidden = answerEncoder.initHidden()
        
        answer_encoder_hidden = repackage_hidden(answer_encoder_hidden)
        question_encoder_1_hidden = repackage_hidden(question_encoder_1_hidden)
        question_encoder_2_hidden = repackage_hidden(question_encoder_2_hidden)
        if type(question_decoder_hidden) == Variable:
            question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        if(type(attn_output) == Variable):
            attn_output = repackage_hidden(attn_output)
        if(type(attention_hidden) == Variable):
            attention_hidden = repackage_hidden(attention_hidden)
        if(type(Attention_Weights) == Variable):
            Attention_Weights = repackage_hidden(Attention_Weights)
        
        current_batch_size = len(batch_input[batch_num]["document_tokens"])
        if current_batch_size != batch_size:
            continue
        
        maxDocLenForBatch = max_document_len
        mask = np.zeros((current_batch_size, maxDocLenForBatch))
        
        for i in range(current_batch_size):
            mask[i,0:batch_input[batch_num]["document_lengths"][i]] = 1
            

        inp = Variable(torch.from_numpy(batch_input[batch_num]["document_tokens"]).long())

        labels = Variable(torch.from_numpy(batch_input[batch_num]["answer_labels_all"])).long()
        if use_cuda:
            labels = labels.cuda()

        optimizer.zero_grad()
        embedded_inp = embedder(inp)
        
        #answer_outputs = Variable(torch.zeros(current_batch_size, max_document_len, hidden_size))
        #answer_tags = Variable(torch.zeros(current_batch_size, max_document_len, 1))
        
        if use_cuda:
            #answer_outputs = answer_outputs.cuda()
            #answer_tags = answer_tags.cuda()
            embedded_inp = embedded_inp.cuda()

        answer_tags, answer_outputs, answer_encoder_hidden = answerEncoder(embedded_inp, answer_encoder_hidden)              

        t_document_mask = Variable(torch.from_numpy(mask)).float()
        if use_cuda:
            t_document_mask = t_document_mask.cuda()
        
        outputs = torch.mul(answer_tags.squeeze(-1),t_document_mask)

        print(outputs.shape, labels.shape)
#         break
        answer_loss = criterion1(outputs, labels.float())
        
        
        
        
        
        answer_mask_1 = Variable(torch.from_numpy(batch_input[batch_num]["answer_labels"])).float().unsqueeze(-1)
        answer_mask_2 = Variable(torch.from_numpy(1 - batch_input[batch_num]["answer_labels"])).float().unsqueeze(-1)
        
        if use_cuda:
            answer_mask_1 = answer_mask_1.cuda()
            answer_mask_2 = answer_mask_2.cuda()
        
        question_encoder_input1 = torch.mul(answer_mask_1, answer_outputs.float())
        question_encoder_input2 = torch.mul(answer_mask_2, answer_outputs.float())
        
        question_encoder_1_outputs , question_encoder_1_hidden = questionEncoder1(question_encoder_input1, question_encoder_1_hidden)
        question_encoder_2_outputs , question_encoder_2_hidden = questionEncoder2(question_encoder_input2, question_encoder_2_hidden)
            
        
        

        question_loss = 0
        question_decoder_hidden = question_encoder_2_hidden
        embedded_inputs = embedder(torch.from_numpy(batch_input[batch_num]["question_input_tokens"]).long())
        output_labels = Variable(torch.from_numpy(batch_input[batch_num]["question_output_tokens"]).long())
        if use_cuda:
            embedded_inputs = embedded_inputs.cuda()
            output_labels = output_labels.cuda()

        for quesL in range(batch_input[batch_num]["question_input_tokens"].shape[1]):
            if use_attention:
                attn_output, Attention_Weights = attention(question_decoder_hidden.squeeze(0).squeeze(0), attention_hidden.squeeze(0), answer_outputs[i])
                decoder_output, attention_hidden = questionDecoder(
                    embedded_inputs[quesL:quesL+1].unsqueeze(1), attn_output)
            else:
                decoder_output, question_decoder_hidden = questionDecoder(
                    embedded_inputs[:,quesL:quesL+1,:],
                    question_decoder_hidden)

            final_output = questionGenerator(decoder_output.squeeze(1))
            
            '''
            idx = np.where (output_labels[:, quesL:quesL+1].squeeze(1).data == END_TOKEN)[0].tolist()
            final_copy = final_output.clone()
            if len(idx):
                print(final_copy.data[idx])
                final_copy.data[idx] = 0
                final_copy.data[idx][:,END_TOKEN] = 1
                print(final_copy[idx])
                print("IDX", idx)
                print("Where:", np.where(final_copy[idx].data == 1)) 
            '''
            print(final_output.shape, output_labels[:, quesL:quesL+1].squeeze(1).shape)
            question_loss += criterion2(final_output,
                                       output_labels[:, quesL:quesL+1].squeeze(1))


        net_loss = answer_loss + question_loss
        net_loss.backward()
        optimizer.step()


        avg_loss+= net_loss.data[0]
        if verboseBatchPrinting:
            print ('Batch: %d \t Epoch : %d\tNet Loss: %.4f \tAnswer Loss: %.4f \tQuestion Loss: %.4f'
                   %(batch_num, epoch, net_loss.data[0], answer_loss.data[0], question_loss.data[0]))



    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f'
                   %(epoch, avg_loss/number_of_batches))

torch.Size([32, 618]) torch.Size([32, 618])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([32])
torch.Size([32, 2000]) torch.Size([3

KeyboardInterrupt: 

In [37]:
outputs.shape

NameError: name 'outputs' is not defined

In [ ]:
final_copy

In [ ]:
idx = [1, 2, 3]
final_output[idx][:,2].shape

In [ ]:
checkpoint = torch.load("/data/ra2630/qgen_base_40k.pt")

In [52]:
answerEncoder2 = AnswerEncoderRNN(input_size = hidden_size, hidden_size=int(hidden_size/2))
questionEncoder2_1 = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionEncoder2_2 = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionDecoder2 = QuestionDecoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionGenerator2 = QuestionGenerationFC(input_size = hidden_size, output_size=reduced_glove.shape[0])
attention2 = AttnDecoderRNN(hidden_size= hidden_size,max_document_len = max_document_len)
optimizer2 = torch.optim.Adam(train_param, 0.01)


#answerEncoder2.load_state_dict(checkpoint["answerEncoder"])
#questionEncoder2.load_state_dict(checkpoint["questionEncoder"])
#questionDecoder2.load_state_dict(checkpoint["questionDecoder"])
#questionGenerator2.load_state_dict(checkpoint["questionGenerator"])
#optimizer2.load_state_dict(checkpoint["optimizer"])

answerEncoder2 = answerEncoder
questionEncoder2_1 = questionEncoder1
questionEncoder2_2 = questionEncoder2
questionDecoder2 = questionDecoder
questionGenerator2 = questionGenerator
attention2 = attention
optimizer2 = optimizer

[Parameter containing:
 -3.7828e-02  6.6429e-02 -5.2511e-02  ...  -8.8349e-02 -3.3487e-02  2.0664e-01
  5.3192e-04 -2.9515e-02 -2.5246e-01  ...   6.4553e-03 -1.2639e-01 -3.4465e-02
 -1.2899e-02  4.3461e-02 -1.0208e-01  ...   5.5923e-02  3.2590e-01  1.3529e-01
                 ...                   ⋱                   ...                
 -1.4746e-01 -1.5048e-03 -1.0250e-01  ...  -1.1302e-01 -1.2740e-01 -2.2424e-01
  8.9877e-02  7.3200e-02 -2.5811e-02  ...   1.1611e-01  1.2277e-01  3.6928e-02
  9.0432e-02  1.2776e-02  1.7478e-02  ...   1.0188e-01  6.7899e-03  1.3490e-01
 [torch.cuda.FloatTensor of size 450x300 (GPU 0)], Parameter containing:
  1.0948e-01 -5.2877e-03 -1.4122e-01  ...  -6.3701e-02 -4.3667e-03  1.4487e-02
  1.7246e-02  2.1329e-01 -1.3400e-01  ...   1.4544e-02  1.1322e-01  9.0475e-02
  1.7838e-02 -2.2348e-01 -4.9929e-02  ...   3.1174e-02  2.8664e-02 -1.1325e-01
                 ...                   ⋱                   ...                
 -3.7944e-02  7.7023e-02 -1.7932e-0

In [ ]:
def printQuestion(batch_num, example_num):
    for i in batch_input[batch_num]["question_input_tokens"][example_num]:
        print(look_up_token_reduced(i), end = ' ', sep = ' ')
    print("")
        
def printAnswer(batch_num, example_num):
    for i in np.where(batch_input[batch_num]["answer_labels"][example_num]==1)[0]:
        print(look_up_token_reduced(batch_input[batch_num]["document_tokens"][example_num][i]))
    print("")
        
def printContext(batch_num, example_num):
    for i in batch_input[batch_num]["document_tokens"][example_num]:
        print(look_up_token_reduced(i), end = ' ', sep = ' ')
    print("")

In [ ]:
batch_num = 3
example_num=3
print("Context : ")
printContext(batch_num, example_num)
print("Question : ")
printQuestion(batch_num, example_num)
print("Answer : ")
printAnswer(batch_num, example_num)

In [ ]:
question_token_embedding.shape, question_decoder_hidden_new.shape, final_output.shape, decoder_output.shape

In [129]:
question_token = START_TOKEN
question_token_embedding = embedder(Variable(torch.from_numpy(np.array([question_token])).long())).cuda()
question_decoder_hidden_new =  question_encoder_2_hidden.clone()
# question_decoder_hidden_new =   questionEncoder2.initHidden()
qLen = 0
b = []
while qLen <= 20:
    decoder_output, question_decoder_hidden_new = questionDecoder(
        question_token_embedding.expand(32,1,300),
        question_decoder_hidden_new)
    final_output = questionGenerator(decoder_output)
    question_token = np.argmax(final_output.data, axis=2)[1][0]
#     w = [np.argmax(final_output.data, axis=2)[i][0] for i in range(batch_size)]
    #b.append([look_up_token_reduced(x) for x in w])
    print(look_up_token_reduced(question_token))
    question_token_embedding = embedder(Variable(torch.from_numpy(np.array([question_token])).long())).cuda()
    qLen=qLen + 1

what
territories
were
<UNK>
by
<UNK>
in
<UNK>
?
<END>
<END>
<END>
<END>
<END>
<END>
<END>
<END>
<END>
<END>
<END>
<END>


In [111]:
final_output.shape

torch.Size([32, 1, 2000])

In [110]:
np.argmax(final_output.data, axis=2)


    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
    2
[torch.LongTensor of size 32x1]

In [78]:
np.argmax(final_output.data, axis=1).shape

torch.Size([32, 2000])

In [82]:
np.argmax(final_output.data, axis = 2)[0]


 44
[torch.LongTensor of size 1]

In [49]:
def prepareSetForInferenceFromBatch(batch_number):
    data_dict = batch_input[batch_number]
    data_dict['answer_mask_1'] = data_dict['answer_labels']
    data_dict['answer_mask_2'] = 1 - data_dict['answer_mask_1']
    return data_dict

In [64]:
def prepareSetForInference(context, answer, maxDocLength):
    
    passage = word_tokenize(context.lower())
    doc_len = len(passage)
    
    ans_condition = word_tokenize(answer.lower())
    
    ans_len = len(ans_condition)
        
    start,end = find_sub_list(ans_condition,passage)

    if start == -1:
            print("Couldn' Find answer in text, Please try again !!")
            return None
    
    
    document_token = np.full((1, maxDocLength), END_TOKEN, dtype=np.int32)
    document_length = np.zeros(1, dtype=np.int32)
    answer_mask_1 = np.zeros((1,maxDocLength), dtype=np.int32)
    answer_mask_2 = np.zeros((1,maxDocLength), dtype=np.int32)
    answer_length = np.zeros(1, dtype=np.int32)
    
    document_length[0] = doc_len
    answer_length[0] = ans_len
    answer_mask_1[0,start:end] = 1
    answer_mask_2 = 1 - answer_mask_1
    
    
    for i in range(doc_len):
        document_token[:,i] = look_up_word_reduced(passage[i])
    
    document_token = document_token.repeat(batch_size, axis = 0)
    document_length = document_length.repeat(batch_size, axis = 0)
    answer_mask_1 = answer_mask_1.repeat(batch_size, axis = 0)
    answer_mask_2 = answer_mask_2.repeat(batch_size, axis = 0)
    answer_length = answer_length.repeat(batch_size, axis = 0)
    
    
    return {
        "document_tokens" : document_token,
        "document_length" : document_length,
        "answer_mask_1" : answer_mask_1,
        "answer_mask_2" : answer_mask_2,
        "answer_length" : answer_length,
    }

In [66]:
def inference2(context, answer, maxDocLen):

    #data_dict = prepareSetForInference(context, answer, maxDocLen)
    data_dict = prepareSetForInferenceFromBatch(2)
    use_attention = False
    use_cuda = True

    answer_encoder_hidden_inf = answerEncoder2.hiddenState
    question_encoder_hidden_1_inf = questionEncoder2_1.hiddenState
    question_encoder_hidden_2_inf = questionEncoder2_2.hiddenState
    question_decoder_hidden_inf = questionDecoder2.hiddenState

    if use_attention:
        attention_hidden_inf = attention.hidden_state




    inp = Variable(torch.from_numpy(data_dict["document_tokens"]).long())
    labels = Variable(torch.from_numpy(data_dict["answer_mask_1"])).long() #Let's see if we want to use or not
    if use_cuda:
        labels = labels.cuda()

    embedded_inp = embedder(inp)

    if use_cuda:
        embedded_inp = embedded_inp.cuda()


    answer_encoder_hidden_inf = repackage_hidden(answer_encoder_hidden_inf)
    answer_tags, answer_outputs, answer_encoder_hidden_inf = answerEncoder(embedded_inp, answer_encoder_hidden_inf)



    if use_cuda:
        answer_outputs = answer_outputs.cuda()
        answer_tags = answer_tags.cuda() 

    ####USE ANSWER_TAGS T CONDITION QUESTIONS RHATHRE THAN PASSING ANSWERS

    answer_mask_1 = Variable(torch.from_numpy(data_dict["answer_mask_1"])).float().unsqueeze(-1)
    answer_mask_2 = Variable(torch.from_numpy(data_dict["answer_mask_2"])).float().unsqueeze(-1)

    if use_cuda:
        answer_mask_1 = answer_mask_1.cuda()
        answer_mask_2 = answer_mask_2.cuda()

    question_encoder_input1 = torch.mul(answer_mask_1, answer_outputs.float())
    question_encoder_input2 = torch.mul(answer_mask_2, answer_outputs.float())

    if use_cuda:
        question_encoder_input1 = question_encoder_input1.cuda()
        question_encoder_input2 = question_encoder_input2.cuda()

    question_encoder_1_outputs , question_encoder_1_hidden = questionEncoder1(question_encoder_input1, question_encoder_hidden_1_inf)
    question_encoder_2_outputs , question_encoder_2_hidden = questionEncoder2(question_encoder_input2, question_encoder_hidden_2_inf)



    maxGenQuesLen = 20
    currGenQuestionLen = 0

    current_question_token = START_TOKEN
    questionGenerated = []
    while currGenQuestionLen < maxGenQuesLen:
        question_token_embedding = embedder(Variable(torch.from_numpy(np.array([current_question_token])).long()))
        if use_cuda:
            question_token_embedding = question_token_embedding.cuda()

        if use_attention:
            attn_output, Attention_Weights = attention(question_decoder_hidden.squeeze(0).squeeze(0), attention_hidden.squeeze(0), answer_outputs[i])
            decoder_output, attention_hidden = questionDecoder(
                embedded_inputs[quesL:quesL+1].unsqueeze(1), attn_output)
        else:
            decoder_output, question_decoder_hidden_inf = questionDecoder2(
                question_token_embedding.expand(32,1,300),
                question_decoder_hidden_inf)

        final_output = questionGenerator2(decoder_output.squeeze(1))
        current_question_token = np.argmax(final_output.data, axis = 1)[0]
        questionGenerated.append(look_up_token_reduced(current_question_token))
        currGenQuestionLen += 1

    return questionGenerated

In [67]:
qGenerated = inference2("Some topic is being written here", "some topic", max_document_len)

Variable containing:

Columns 0 to 12 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 13 to 25 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 26 to 38 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 39 to 51 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 52 to 64 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 65 to 77 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 78 to 90 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 91 to 103 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 104 to 116 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 117 to 129 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Colu

In [ ]:
batch_num = 3
example_num = 0
printQuestion(batch_num, example_num)
printAnswer(batch_num, example_num)
printContext(batch_num, example_num)

In [ ]:
' '.join(X_train_comp_all_shuffled[3*32 + 20])

In [ ]:
' '.join(X_train_ans_all_shuffled[3*32 + 20])

In [ ]:
' '.join(Y_train_ques_all_shuffled[3*32 + 20])

In [ ]:
inference2(' '.join(X_train_comp_all_shuffled[3*32 + 20]), ' '.join(X_train_ans_all_shuffled[3*32 + 20]), max_document_len)

In [ ]:
batch_input[0]['question_input_tokens'][0], batch_input[0]['question_output_tokens'][0]